In [ ]:
#scraping bookings.com
import requests
from bs4 import BeautifulSoup as Soup
import pandas as pd
df = pd.DataFrame()
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'}
for pages in range(0, 39):
    req = requests.get(f"https://www.booking.com/searchresults.en-gb.html?label=gog235jc-1FCAIobDgOSAlYA2hsiAEBmAEJuAEXyAEM2AEB6AEB-AECiAIBqAIDuAKw3J2jBsACAdICJDNlOWY5MDE2LWRkNzMtNDBhZS1iMjQ2LTQ5ZTk2ZDJmYjU5ZdgCBeACAQ&sid=78a835dba3d7708eeae1b91933ba50ae&aid=356980&checkin=2023-06-01&checkout=2023-06-02&dest_id=-2106102&dest_type=city&srpvid=c0886213ab84013a&track_hp_back_button=1&offset={pages*25}", headers=headers).text
    soup = Soup(req, 'html.parser')
    apts = soup.find_all("div", {"class": "d20f4628d0"})
    l = []
    obj = {}
    for a in range(0, len(apts)):
        try:
            obj["pricing"] = apts[a].find("span", {"class": "fcab3ed991 fbd1d3018c e729ed5ab6"}).text
        except:
            obj["pricing"] = None
        try:
            obj["Distance"] = apts[a].find("span", {"class": "cb5ebe3ffb"}).text
        except:
            obj["Distance"] = None
        try:
            ap1=apts[a].find('a',href=True)
            link = ap1['href']
            req1 = requests.get(link, headers=headers).text
            soup2 = Soup(req1, 'html.parser')
            obj["amenities"] = soup2.find("div", {"class": "e5e0727360"}).text
        except:
            obj["amenities"] = None
        try:
            obj["ratings"] = apts[a].find("div", {"class": "b5cd09854e d10a6220b4"}).text
        except:
            obj["ratings"] = None
        try:
            obj["type"] = apts[a].find("span", {"class": "df597226dd"}).text
        except:
            obj["type"] = None
        try:
            obj["location"] = apts[a].find("span", {"class": "f4bd0794db b4273d69aa"}).text
        except:
            obj["location"] = None
    
    
        l.append(obj)
        obj = {}
        for i in l:
            price = i["pricing"]
            distance = i["Distance"]
            amenities = i["amenities"]
            ratings = i["ratings"]
            room_type=i["type"]
            location=i["location"]
            df = df.append({"price": price, "location":location,"distance": distance,"amenities": amenities, "ratings": ratings,"type":room_type}, ignore_index=True)
# dataCleaning
df["price"] = df["price"].str.replace(r"₹", "")
df["price"] = df["price"].str.replace(r" ", "")
df["price"] = df["price"].str.replace(r",", "")
df["price"] = df["price"].str.strip() 
df['price'] = pd.to_numeric(df['price'])
df['ratings'] = pd.to_numeric(df['ratings'], errors='coerce')
df['ratings'] = df['ratings'].fillna(df['ratings'].mean())
dfd=dfd.drop_duplicates(ignore_index=True)
df.to_csv('test.csv')
df   

In [ ]:
# Web Scrapping OYO Working Code
import requests
from bs4 import BeautifulSoup as Soup
import pandas as pd
def delhi():
    df = pd.DataFrame()
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}
    for pages in range(1, 6):
        req = requests.get(
            f"https://www.oyorooms.com/hotels-in-delhi/?page={pages}", headers=headers).text
        soup = Soup(req, 'html.parser')
        apts = soup.find_all(
            "div", {"class": "oyo-row oyo-row--no-spacing listingHotelDescription"})
        l = []
        obj = {}
        apts1 = soup.find_all(itemprop='url')
        for a in range(0, len(apts)):
            try:
                obj["pricing"] = apts[a].find(
                    "span", {"class": "listingPrice__finalPrice"}).text
            except:
                obj["pricing"] = None
            try:
                obj["Distance"] = apts[a].find(
                    "span", {"class": "listingHotelDescription__distanceText"}).text
            except:
                obj["Distance"] = None
            try:
                link = apts1[a].get('content')
                req1 = requests.get(link, headers=headers).text
                soup2 = Soup(req1, 'html.parser')
                ap=soup2.find_all("div", {"class": "c-riklip"})
                stri=''
                for b in range(0,len(ap)):
                    if "striked" not in ap[b].get("class", []):
                        stri += ap[b].find("div", {"class": "c-12w6zty"}).text + " "
                obj["amneties"] = stri.strip()
            except:
                obj["amneties"] = None
            try:
                obj["ratings"] = apts[a].find(
                    "div", {"class": "hotelRating"}).text
            except:
                obj["ratings"] = None

            l.append(obj)
            obj = {}
        for i in l:
            price = i["pricing"]
            distance = i["Distance"]
            amneties = i["amneties"]
            ratings = i["ratings"]
            df = df.append({"price": price, "distance": distance,
                           "amneties": amneties, "ratings": ratings}, ignore_index=True)
# dataCleaning
    df["price"] = df["price"].str.replace(r"₹", "")
    df["ratings"] = df["ratings"].str.replace(r"₹", "")
    df['price'] = pd.to_numeric(df['price'])
    df['distance'] = df['distance'].str.replace(r"km", "")
    df['distance'] = pd.to_numeric(df['distance'])
    df[['ratings', 'last_name', 'y', 'x']
       ] = df['ratings'].str.split(' ', expand=True)
    df = df.drop(columns=['last_name', 'y', 'x'], axis=1)
    df['ratings'] = pd.to_numeric(df['ratings'], errors='coerce')
    df['ratings'] = df['ratings'].fillna(df['ratings'].mean())
    return df
df = delhi()
df['price'] = df['price'].fillna(df['price'].mean())
df.to_csv('test1.csv')